com_error: (-2147352567, '예외가 발생했습니다.', (0, 'Microsoft PowerPoint', "Slides.Item : Integer out of range. 1 is not in Index's valid range of 1 to 0.", '', 0, -2147188160), None)

In [9]:
import win32com.client

# 이미 실행 중인 PowerPoint 애플리케이션에 연결
try:
    ppt = win32com.client.GetActiveObject("PowerPoint.Application")
    print("기존 PowerPoint 애플리케이션에 연결됨")
except:
    # 실행 중인 PowerPoint가 없으면 새로 시작
    ppt = win32com.client.Dispatch("PowerPoint.Application")
    ppt.Visible = True
    print("새 PowerPoint 애플리케이션 시작됨")

# 현재 활성화된 프레젠테이션 가져오기
presentation = ppt.ActivePresentation
print(f"프레젠테이션 '{presentation.Name}' 접근됨")

# 현재 선택된 슬라이드 또는 특정 슬라이드에 접근
success_method = None  # 성공한 방법을 추적하는 변수
try:
    slide = ppt.ActiveWindow.View.Slide
    print(f"현재 활성 슬라이드 {slide.SlideIndex}번에 접근됨")
except:
    # 특정 슬라이드에 접근 (예: 첫 번째 슬라이드)
    slide_index = 1
    slide = presentation.Slides(slide_index)
    print(f"첫 번째 슬라이드에 접근됨")

# 노트 텍스트 설정하기 - 여러 방법 시도
notes_text = "dlalal"

# 방법 1: 노트 페이지에 직접 접근
try:
    slide.NotesPage.Shapes.Placeholders(2).TextFrame.TextRange.Text = notes_text
    print("방법 1 성공: NotesPage.Shapes.Placeholders(2) 사용")
    success_method = "방법 1: NotesPage.Shapes.Placeholders(2)"
except Exception as e:
    print(f"방법 1 실패: {str(e)}")
    
    # 방법 2: 다른 인덱스의 Shape 시도
    try:
        slide.NotesPage.Shapes(2).TextFrame.TextRange.Text = notes_text
        print("방법 2 성공: NotesPage.Shapes(2) 사용")
        success_method = "방법 2: NotesPage.Shapes(2)"
    except Exception as e:
        print(f"방법 2 실패: {str(e)}")
        
        # 방법 3: 모든 Shape 탐색
        try:
            shape_found = False
            for shape_idx in range(1, slide.NotesPage.Shapes.Count + 1):
                try:
                    shape = slide.NotesPage.Shapes(shape_idx)
                    if hasattr(shape, 'TextFrame') and hasattr(shape.TextFrame, 'TextRange'):
                        shape.TextFrame.TextRange.Text = notes_text
                        print(f"방법 3 성공: Shape {shape_idx}에 텍스트 설정됨")
                        success_method = f"방법 3: Shape {shape_idx}"
                        shape_found = True
                        break
                except Exception as inner_e:
                    print(f"  Shape {shape_idx} 처리 중 오류: {str(inner_e)}")
            
            if not shape_found:
                raise Exception("적합한 텍스트 프레임을 가진 Shape을 찾지 못함")
                
        except Exception as e:
            print(f"방법 3 실패: {str(e)}")
            
            # 방법 4: 노트 속성 직접 사용
            try:
                slide.Notes = notes_text
                print("방법 4 성공: slide.Notes 속성 사용")
                success_method = "방법 4: slide.Notes 속성"
            except Exception as e:
                print(f"방법 4 실패: {str(e)}")

# 결과 요약
if success_method:
    print("\n===== 결과 요약 =====")
    print(f"슬라이드 {slide.SlideIndex}번의 노트에 텍스트 설정 성공!")
    print(f"성공한 방법: {success_method}")
else:
    print("\n===== 결과 요약 =====")
    print("모든 방법 실패: 슬라이드 노트에 텍스트를 설정하지 못했습니다.")

print("\n작업 완료")

기존 PowerPoint 애플리케이션에 연결됨
프레젠테이션 'test_ppt_원본.pptx' 접근됨
현재 활성 슬라이드 3번에 접근됨
방법 1 성공: NotesPage.Shapes.Placeholders(2) 사용

===== 결과 요약 =====
슬라이드 3번의 노트에 텍스트 설정 성공!
성공한 방법: 방법 1: NotesPage.Shapes.Placeholders(2)

작업 완료
